In [29]:
import tensorflow as tf

# Configurar la GPU antes de inicializar TensorFlow
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices) 


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# WSL FUNCIONES GPU CON LIBRERIAS SIMILARES PARA DATASCIENTIST
import cudf
import cuml
import cupy
import dask_cudf
from cuml.decomposition import PCA
import glob
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask.dataframe as dd
import dask_cudf as dd



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import funciones_ML as bt
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder, Normalizer, RobustScaler
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay,classification_report,r2_score,RocCurveDisplay,confusion_matrix, accuracy_score,recall_score,f1_score,precision_score,precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.pipeline import Pipeline
"""from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2, l1, l1_l2
from sklearn.cluster import KMeans
#from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras import regularizers
from sklearn.compose import ColumnTransformer
from tensorflow.keras.losses import CategoricalCrossentropy, sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau"""
import multiprocessing as mp


# Configurar pandas para mostrar todas las filas y columnas
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Código para los tamaños de las fuentes y los "ticks" de los ejes:
plt.rc('font', size=6)
plt.rc('axes', labelsize=10, titlesize=10)
plt.rc('legend', fontsize=10)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [6]:
df_ml=cudf.read_csv(file_path)

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /opt/conda/conda-bld/work/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [5]:
# Ruta al archivo CSV gigante
file_path = "/mnt/d/Cursos/REPOSITORIOS/DATASET/malware_total/original/df_one_hot_full_ml_mm.csv"

In [ ]:


# Lista para almacenar los DataFrames
df_list = []
# Leer los archivos CSV en paralelo
for file in glob.glob(file_path):
    df_list.append(dd.from_cudf(cudf.read_csv(file), npartitions=4))

# dividir DataFrame origen  en 4
full_df = [df.to_delayed() for df in df_list]

# Verificar la forma del DataFrame completo
print(full_df.shape)


In [4]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25011003 entries, 0 to 25011002
Columns: 288 entries, Unnamed: 0 to history_nan
dtypes: float64(287), int64(1)
memory usage: 53.7 GB


In [13]:
df_ml.info()

<class 'dask_cudf.core.DataFrame'>
Columns: 288 entries, Unnamed: 0 to history_nan
dtypes: float64(287), int64(1)

In [24]:
# Ruta al archivo CSV gigante
file_path = "/mnt/d/Cursos/REPOSITORIOS/DATASET/malware_total/original/df_one_hot_full_ml_mm.csv"

In [34]:

def dividir_dataset_en_lotes(file_path, tamano_lote):
    """
    Divide un dataset en lotes utilizando cuDF.
    
    Args:
    - file_path: Ruta al archivo CSV.
    - tamano_lote: Tamaño de cada lote.
    
    Returns:
    - Una lista de DataFrames cuDF, donde cada DataFrame representa un lote del dataset.
    """
    # Leer el archivo CSV completo en un DataFrame cuDF
    df_completo = cudf.read_csv(file_path)
    
    # Calcular el número total de filas en el DataFrame
    num_filas = len(df_completo)
    
    # Calcular el número total de lotes
    num_lotes = (num_filas + tamano_lote - 1) // tamano_lote
    
    # Dividir el DataFrame en lotes
    lotes = []
    for i in range(num_lotes):
        inicio = i * tamano_lote
        fin = min((i + 1) * tamano_lote, num_filas)
        lote = df_completo.iloc[inicio:fin]
        lotes.append(lote)
    
    return lote

In [ ]:

file_path = file_path  # Ruta al archivo CSV
tamano_lote = 2500000  # Tamaño de cada lote

# Dividir el dataset en lotes
df_lotes = dividir_dataset_en_lotes(file_path, tamano_lote)



In [ ]:
# Definir el prefijo para los nombres de los archivos de salida
output_file_prefix = "lote_"

# Iterar sobre los dataframes y guardar cada uno en un archivo CSV
for i, df_lote in enumerate(df_lotes):
    output_file_path = f"{output_file_prefix}{i}.csv"  # Nombre del archivo de salida
    df_lote.to_csv(output_file_path, index=False)  # Guardar el dataframe en el archivo CSV

print("¡Guardado exitoso!")

In [ ]:
#PCA INVREMENTAL
# Carga el dataset en un array de NumPy
data = np.load("ruta/al/dataset.npy")

# Divide el dataset en batches de 100.000 filas
batches = np.array_split(data, 250)

# Crea un objeto PCA incremental con 10 componentes principales
pca = IncrementalPCA(n_components=10, whiten=True, copy=True)

for batch in batches:
    # Actualiza el objeto PCA con el batch actual
    pca.partial_fit(batch)

# Obtén las componentes principales
components = pca.components_

# Obtén las varianzas explicadas por cada componente principal
variances = pca.explained_variance_


In [ ]:
# Guarda las componentesprincipales en un archivo
np.save("ruta/a/las/componentes.npy", components)

# Guarda las varianzas explicadas en un archivo
np.save("ruta/a/las/varianzas.npy", variances)

In [ ]:
import tensorflow as tf

# Supongamos que tienes una lista de 12 DataFrames llamada df_list

# Convertir cada DataFrame a un tensor y almacenarlos en una lista
tensor_list = [tf.constant(df.values) for df in df_list]

# Concatenar los tensores en uno solo
full_tensor = tf.concat(tensor_list, axis=0)

# Supongamos que quieres dividir el tensor en 3 partes
num_splits = 3
split_tensors = tf.split(full_tensor, num_splits)

# Ahora split_tensors contendrá una lista de tensores divididos


In [ ]:
#  número de componentes principales
n_components = 50  
# Inicializar el modelo de PCA
pca = cuml.PCA(n_components=n_components)

# Iterar sobre los chunks del conjunto de datos
for chunk in df_ml.to_delayed():
    # Leer el chunk actual
    data_chunk = chunk.compute()
    
    # Realizar PCA en el chunk actual
    pca.partial_fit(data_chunk)

# Combinar los resultados de PCA de cada chunk
transformed_data = None
for chunk in df_ml.to_delayed():
    data_chunk = chunk.compute()
    transformed_chunk = pca.transform(data_chunk)
    if transformed_data is None:
        transformed_data = transformed_chunk
    else:
        transformed_data = dask_cudf.concat([transformed_data, transformed_chunk])

# Convierte el resultado en un DataFrame para su posterior análisis
df_transformed = transformed_data.compute()

In [ ]:
df_ml.info()

1. PROBAMOS UN RANDONFOREST DE ML CON CONJUNTO DE DATOS TF-DATA


In [18]:

import cudf
from sklearn.model_selection import train_test_split

# Supongamos que tienes 4 DataFrames llamados df1, df2, df3 y df4

# Definir el nombre de la columna objetivo
target_column = "target_column_name"  # Reemplaza "target_column_name" por el nombre real de tu columna objetivo

# Definir listas para almacenar las características (features) y objetivos (targets) de cada DataFrame
all_X = []
all_y = []

# Iterar sobre cada DataFrame para realizar el split
for df in [df1, df2, df3, df4]:
    # Dividir el DataFrame en características y objetivo
    X, y = split_features_and_target(df, target_column)
    # Agregar las características y objetivos a las listas
    all_X.append(X)
    all_y.append(y)

# Concatenar todas las características y objetivos
X = cudf.concat(all_X)
y = cudf.concat(all_y)

# Ahora tienes un único conjunto de características (X) y un único conjunto de objetivos (y) que puedes usar para modelar


array(<MapDataset element_spec=({'': TensorSpec(shape=(128,), dtype=tf.int32, name=None), '0': TensorSpec(shape=(128,), dtype=tf.float32, name=None), '1': TensorSpec(shape=(128,), dtype=tf.float32, name=None), '2': TensorSpec(shape=(128,), dtype=tf.float32, name=None), '3': TensorSpec(shape=(128,), dtype=tf.float32, name=None), '4': TensorSpec(shape=(128,), dtype=tf.float32, name=None), '5': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'proto_icmp': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'proto_tcp': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'proto_udp': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'conn_state_OTH': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'conn_state_REJ': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'conn_state_RSTO': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'conn_state_RSTOS0': TensorSpec(shape=(128,), dtype=tf.float32, name=None), 'conn_state_RSTR': TensorSpec(shape=(128,), dtype

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Definir el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(classification_report(y_pred, y_test))

In [ ]:
importancia=model.feature_importances_

In [ ]:
df_ml.columns

In [ ]:
df_importancia = pd.DataFrame({"importancia":importancia,"Columnas":columnas})

In [ ]:
plt.figure(figsize=(20,6))
plt.bar(df_importancia['Columnas'], df_importancia['importancia'])
plt.xlabel('Variables')
plt.ylabel('Importancia')
plt.show()

SUBMUESTREO DE LA CLASE MALICIOUS QYE ES LA MAYORITARIA PARA CONSEGUIR UN BALANCEO.- SE BALANCEA EL DATASET COMPLETO, DEJANDO LA TRAGET 50/50, PERO PERDIENDO UN 30% DE LOS DATOS

In [ ]:
df_ml.Target.value_counts()

In [ ]:
# Aplica submuestro, ya que la clase minoritaria la ingle yo con datos de la misma columna
X = df_ml.drop('Target', axis=1)  
y = df_ml['Target']    

sampling_strategy="majority"#solo submuestre la clase mayoritaria

ros = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

print(y_resampled.value_counts())

In [ ]:
# Crea una columna temporal
df_temp = pd.concat([X_resampled, y_resampled], axis=1)

# Re-indexa el dataframe
df_temp = df_temp.reset_index(drop=True)  

In [ ]:
sns.histplot(df_temp.Target, bins=8)

In [ ]:
X=df_sca.iloc[:,0:12]
y=df_sca.iloc[:,12]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


MODELO MACHINE LEARENING LOGISTICREGRESSION

In [ ]:
# Definir el modelo logisctic con C para regule el sobreajutes y la funcion sag q es adecuado con muchos datos
model = LogisticRegression(C=10, solver="sag")

# Entrenar el modelo
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(classification_report(y_pred, y_test))

UTILIZANDO TECNICAS DE MACHINE LEARNING TANTO, DESBALANCEADO O BALANCEADA LA TARGET, CON RANDONFOREST Y LOGISTICREGRESION, LOS RESULTADO SOSN MUY BUENOS, PERO PUDIERA HABER UN SOBREAJUSTE, AUNQUE AL LOGISTICREGRESSION SE LE APLICADO UN VALOR MUY AL TO DE C PARA CONTENER EL SOBREAJUSTE, QUIZAS SEA UN MODELO PERFECTO, PERO VAMOS A PROBAR CON TECNICAS DE DEEP LEARNING.<br>

CONTINUAMOS EN NOTEBOOK MALWARE_1_4